# ETL Project  12 January 2022

## Brief description of challenge here
what is ETL, what tools are used, what are the important steps.

## Brief description of the data here
Zillow Real Estate Data (used to be quandl, now Nasdaq), table format

In [1]:
# Setting dependencies, etc.

import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', None)

# for importing data:
import quandl
quandl.ApiConfig.api_key = 'vxSYNGB-9owcVsRx_SEF'

import requests
import statistics
import time

# for communicating to database
import psycopgy2
import io
from sqlalchemy import create_engine

# for displays (may not use)
#import matplotlib.pyplot as plt
#import seaborn as sns
#from pprint import pprint

## Step 1: Data Extraction and preparation

### Extraction: source and comments

The Zillow database comprises three tables: Indicators (## market indicators), Regions (over 78,000!) and Data (market value). 

We extract the data using the quandl library. First, I will extract the data, then look at the data frame to identify any inconsistencies, type errors, etc. 

In [3]:
# Table 1: Data - values for all indicators
# Here, we're restricting the extraction to US records only (region_id = 102001)

zillow_data = quandl.get_table('ZILLOW/DATA', region_id = '102001', paginate = True)
zillow_data.head()

,indicator_id,region_id,date,value
None,,,,
0,ZSFH,102001,2021-07-31,298319.0
1,ZSFH,102001,2021-06-30,292477.0
2,ZSFH,102001,2021-05-31,286907.0
3,ZSFH,102001,2021-04-30,281827.0
4,ZSFH,102001,2021-03-31,277736.0


Here we see the indicator_id, which is a primary key, the region_id (we can eliminate downstream), the date, and market value. Let's see the info and dtypes, ets. 

In [4]:
zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11680 entries, 0 to 11679
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   indicator_id  11680 non-null  object        
 1   region_id     11680 non-null  object        
 2   date          11680 non-null  datetime64[ns]
 3   value         11680 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 365.1+ KB


There are 11680 records.
The date is in datetime64 dtype, and the value is a float, so that's good. No changes necessary. 

In [7]:
# Table 2: Indicators - names and IDs of all indicators
zillow_ind = quandl.get_table('ZILLOW/INDICATORS', paginate = True)
zillow_ind.head()

,indicator_id,indicator,category
None,,,
0,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
1,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
2,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
3,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values


The indicator_id is shared with the "Data" table and makes it amenable to querying. Indicator_id is a four-character classification of a market indicator, which is described in the column "indicator". There are 56 indicators, grouped into one of three categories: home values, rentals, and sales and inventories. 

In [8]:
zillow_ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   indicator_id  56 non-null     object
 1   indicator     56 non-null     object
 2   category      56 non-null     object
dtypes: object(3)
memory usage: 1.4+ KB


In [18]:
# Table 3: Regions - names and IDs of all regions
# Even though we're restricting our attention to the US, there are lots of sub-regions.

zillow_reg = quandl.get_table('ZILLOW/REGIONS', paginate = True)
zillow_reg.head()

,region_id,region_type,region
None,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin
1,99998,zip,98846; WA; Okanogan County; Pateros
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades
3,99996,zip,98844; WA; Okanogan County; Oroville
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo


In [19]:
zillow_reg.tail()

,region_id,region_type,region
None,,,
79244,100000,zip,98848; WA; Moses Lake; Grant County; Quincy
79245,10000,city,Bloomington; MD; Garrett County
79246,1000,county,Echols County; GA; Valdosta
79247,100,county,Bibb County; AL; Birmingham-Hoover
79248,10,state,Colorado; CO


In [11]:
zillow_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79249 entries, 0 to 79248
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   region_id    79249 non-null  object
 1   region_type  79249 non-null  object
 2   region       79249 non-null  object
dtypes: object(3)
memory usage: 1.8+ MB


In [20]:
list(zillow_reg['region_type'].unique())

['zip', 'city', 'county', 'neigh', 'state', 'metro']

There's a lot of variety in the information in the 'region' column. For the database, each column has to have one type of entry. The solution is to create additional columns for each one of the six region types (listed above). The data from region will be parsed into those columns, then deleted. 

## Preparation
checking data types, removing uninformative characters, parsing info from one column into many

In [21]:
# Making the necessary additional columns (object Dtype)

zillow_reg['zip'] = ''
zillow_reg['state'] = ''
zillow_reg['neigh'] = ''
zillow_reg['county'] = ''
zillow_reg['city'] = ''

zillow_reg.head()

,region_id,region_type,region,zip,state,neigh,county,city
None,,,,,,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin,,,,,
1,99998,zip,98846; WA; Okanogan County; Pateros,,,,,
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades,,,,,
3,99996,zip,98844; WA; Okanogan County; Oroville,,,,,
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo,,,,,


In [22]:
# Parsing out information for the different region types. Using a double loop. A series of conditionals based on evaluating the contents between splitting character, The semicolon ';' character is what defines splitting fields
# Starting with zipcode. Go l-r down the string. First, look for a set of digits. If they exist, put into zip column. Next, look for a set of 3 characters (state), save as state, etc.

zipcode = zillow_reg[zillow_reg['region_type'] == 'zip']['region'].str.split(';', n=4, expand = True)
for i in range(5):
    for j in zipcode.index:
        if str(zipcode[i][j]).isdigit():
            zillow_reg['zip'][j] = zipcode[i][j]
        elif len(str(zipcode[i][j])) == 3:
            zillow_reg['state'][j] = zipcode[i][j]
        elif str(zipcode[i][j]).find('-') != -1:  ## comment on '-'
            zillow_reg['neigh'][j] = zipcode[i][j]
        elif str(zipcode[i][j]).find(' County') != -1:
            zillow_reg['county'][j] = zipcode[i][j]
        elif str(zipcode[i][j]).find(' Borough') != -1:
            zillow_reg['county'][j] = zipcode[i][j] #classifying boroughs as counties
        else:
            zillow_reg['city'][j] = zipcode[i][j]
               
        

In [23]:
zillow_reg.head()

,region_id,region_type,region,zip,state,neigh,county,city
None,,,,,,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin,98847,WA,,Chelan County,Peshastin
1,99998,zip,98846; WA; Okanogan County; Pateros,98846,WA,,Okanogan County,None
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades,98845,WA,,Douglas County,Palisades
3,99996,zip,98844; WA; Okanogan County; Oroville,98844,WA,,Okanogan County,None
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo,98843,WA,,Douglas County,Orondo


In [24]:
zillow_reg.tail()

,region_id,region_type,region,zip,state,neigh,county,city
None,,,,,,,,
79244,100000,zip,98848; WA; Moses Lake; Grant County; Quincy,98848,WA,,Grant County,Quincy
79245,10000,city,Bloomington; MD; Garrett County,,,,,
79246,1000,county,Echols County; GA; Valdosta,,,,,
79247,100,county,Bibb County; AL; Birmingham-Hoover,,,,,
79248,10,state,Colorado; CO,,,,,


So far, so good. Now, follow same procedure for remaining region types. 

In [30]:
# region type: city

city = zillow_reg[zillow_reg['region_type'] == 'city']['region'].str.split(';', n=4, expand = True)
for i in range(0,1):
    for j in city.index: 
        if str(city[i][j]).isdigit():
            zillow_reg['zip'][j] = city[i][j]
        elif len(str(city[i][j])) == 3:
            zillow_reg['state'][j] = city[i][j]
        elif str(city[i][j]).find('-') != -1:  
            zillow_reg['neigh'][j] = city[i][j]
        elif str(city[i][j]).find(' County') != -1:
            zillow_reg['county'][j] = city[i][j]
        elif str(city[i][j]).find(' Borough') != -1:
            zillow_reg['county'][j] = city[i][j] 
        else:
            zillow_reg['city'][j] = city[i][j]

In [31]:
# region type: neighborhood ('neigh')
# there's not many of them

nhood = zillow_reg[zillow_reg['region_type'] == 'neigh']['region'].str.split(';', n = 4, expand = True)
for i in range (4):
    for j in nhood.index: 
        if str(nhood[i][j]).isdigit():
            zillow_reg['zip'][j] = nhood[i][j]
        elif len(str(nhood[i][j])) == 3:
            zillow_reg['state'][j] = nhood[i][j]
        elif str(nhood[i][j]).find('-') != -1:  
            zillow_reg['neigh'][j] = nhood[i][j]
        elif str(nhood[i][j]).find(' County') != -1:
            zillow_reg['county'][j] = nhood[i][j]
        elif str(nhood[i][j]).find(' Borough') != -1:
            zillow_reg['county'][j] = nhood[i][j] 
        else:
            zillow_reg['city'][j] = nhood[i][j]

In [44]:
nhood

,0,1,2,3,4
None,,,,,
625,Hunter Army Airfield,GA,Savannah,Chatham County,None
2308,Lakeshore,LA,Monroe,Ouachita Parish,None
2545,North Grosvenordale,CT,Worcester,Windham County,Thompson
2618,Mount Lebanon,PA,Pittsburgh,Allegheny County,None
3119,Westmere,NY,Albany-Schenectady-Troy,Albany County,Guilderland
...,...,...,...,...,...
78862,Central,CA,San Francisco-Oakland-Hayward,Alameda County,Alameda
78864,Bjerremark,AK,Fairbanks,Fairbanks North Star Borough,None
78865,Birchwood,AK,Anchorage,Anchorage Borough,None


In [32]:
zillow_reg.tail()

,region_id,region_type,region,zip,state,neigh,county,city
None,,,,,,,,
79244,100000,zip,98848; WA; Moses Lake; Grant County; Quincy,98848,WA,,Grant County,Quincy
79245,10000,city,Bloomington; MD; Garrett County,,MD,,Garrett County,Bloomington
79246,1000,county,Echols County; GA; Valdosta,,,,,
79247,100,county,Bibb County; AL; Birmingham-Hoover,,,,,
79248,10,state,Colorado; CO,,,,,


In [33]:
city

,0,1,2,3
None,,,,
10,Carrsville,VA,Virginia Beach-Norfolk-Newport News,Isle of Wight County
20,Birchleaf,VA,Big Stone Gap,Dickenson County
56,Wright,KS,Dodge City,Ford County
124,Weston,CT,Bridgeport-Stamford-Norwalk,Fairfield County
168,South Wilmington,IL,Chicago-Naperville-Elgin,Grundy County
...,...,...,...,...
79138,Audubon,IA,Audubon County,None
79168,Bound Brook,NJ,New York-Newark-Jersey City,Somerset County
79198,Chanute,KS,Neosho County,None


In [47]:
# region type: county

county = zillow_reg[zillow_reg['region_type'] == 'county']['region'].str.split(';', n = 4, expand = True)
for i in range (2):
    for j in county.index: 
        if str(county[i][j]).isdigit():
            zillow_reg['zip'][j] = county[i][j]
        elif len(str(county[i][j])) == 3:
            zillow_reg['state'][j] = county[i][j]
        elif str(county[i][j]).find('-') != -1:  
            zillow_reg['neigh'][j] = county[i][j]
        elif str(county[i][j]).find(' County') != -1:
            zillow_reg['county'][j] = county[i][j]
        elif str(county[i][j]).find(' Borough') != -1:
            zillow_reg['county'][j] = county[i][j] 
        else:
            zillow_reg['city'][j] = county[i][j]

In [48]:
county

,0,1,2
None,,,
94,Durham County,NC,Durham-Chapel Hill
169,Duplin County,NC,None
246,Dubois County,IN,Jasper
401,Donley County,TX,None
588,Dimmit County,TX,None
...,...,...,...
79014,Elmore County,AL,Montgomery
79065,Elbert County,GA,None
79148,Elbert County,CO,Denver-Aurora-Lakewood


In [34]:
# region type: state

state = zillow_reg[zillow_reg['region_type'] == 'state']['region'].str.split(';', n = 1, expand = True)
for i in range (2):
    for j in state.index: 
        if str(state[i][j]).isdigit():
            zillow_reg['zip'][j] = state[i][j]
        elif len(str(state[i][j])) == 3:
            zillow_reg['state'][j] = state[i][j]
        elif str(state[i][j]).find('-') != -1:  
            zillow_reg['neigh'][j] = state[i][j]
        elif str(state[i][j]).find(' County') != -1:
            zillow_reg['county'][j] = state[i][j]
        elif str(state[i][j]).find(' Borough') != -1:
            zillow_reg['county'][j] = state[i][j] 
        else:
            zillow_reg['city'][j] = state[i][j]

In [37]:
state


,0,1
None,,
7326,California,CA
15748,Arizona,AZ
34210,Wyoming,WY
34702,West Virginia,WV
35518,Wisconsin,WI
35519,Arkansas,AR
36398,Washington,WA
37100,Vermont,VT
37821,Virginia,VA


In [35]:
# region type: metro

metro = zillow_reg[zillow_reg['region_type'] == 'metro']['region'].str.split(';', n = 1, expand = True)
met = metro[0].str.split(',', n = 2, expand = True) # there's a second delimiter to deal with
for i in range (2):
    for j in met.index: 
        if str(met[i][j]).isdigit():
            zillow_reg['zip'][j] = met[i][j]
        elif len(str(met[i][j])) == 3:
            zillow_reg['state'][j] = met[i][j]
        elif str(met[i][j]).find('-') != -1:  
            zillow_reg['neigh'][j] = met[i][j]
        elif str(met[i][j]).find(' County') != -1:
            zillow_reg['county'][j] = met[i][j]
        elif str(met[i][j]).find(' Borough') != -1:
            zillow_reg['county'][j] = met[i][j] 
        else:
            zillow_reg['city'][j] = met[i][j]

In [36]:
met

,0,1
None,,
23049,Zapata,TX
23050,Wisconsin Rapids-Marshfield,WI
23051,Winnemucca,NV
23052,Weatherford,OK
23053,Vineyard Haven,MA
...,...,...
51654,Ada,OK
51656,Abilene,TX
51657,Aberdeen,WA


In [50]:
# Looking at the data frame:
zillow_reg.tail(15)

,region_id,region_type,region,zip,state,neigh,county,city
None,,,,,,,,
79234,10001,city,Blountsville; AL; Birmingham-Hoover; Blount County,,AL,Birmingham-Hoover,Blount County,Blountsville
79235,100009,zip,98858; WA; Wenatchee; Douglas County; Waterville,98858,WA,,Douglas County,Waterville
79236,100008,zip,98857; WA; Moses Lake; Grant County; Warden,98857,WA,,Grant County,Warden
79237,100007,zip,98856; WA; Okanogan County; Twisp,98856,WA,,Okanogan County,None
79238,100006,zip,98855; WA; Okanogan County; Tonasket,98855,WA,,Okanogan County,None
79239,100005,zip,98853; WA; Moses Lake; Grant County,98853,WA,,Grant County,None
79240,100004,zip,98852; WA; Okanogan County; Winthrop,98852,WA,,Okanogan County,None
79241,100003,zip,98851; WA; Moses Lake; Grant County; Soap Lake,98851,WA,,Grant County,Soap Lake
79242,100002,zip,98850; WA; Wenatchee; Douglas County; Rock Island,98850,WA,,Douglas County,Rock Island


In [52]:
zillow_reg.head()

,region_id,region_type,region,zip,state,neigh,county,city
None,,,,,,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin,98847,WA,,Chelan County,Peshastin
1,99998,zip,98846; WA; Okanogan County; Pateros,98846,WA,,Okanogan County,None
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades,98845,WA,,Douglas County,Palisades
3,99996,zip,98844; WA; Okanogan County; Oroville,98844,WA,,Okanogan County,None
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo,98843,WA,,Douglas County,Orondo


This is imperfect; there are plenty of cities not being parsed out correctly. I'm moving forward with the exercise and will not use 'city' in a query.

In [53]:
# dropping 'region' column from table:

zillow_reg.drop('region', axis = 1, inplace = True)

In [54]:
zillow_reg[zillow_reg['region_type'] == 'zip']

,region_id,region_type,zip,state,neigh,county,city
None,,,,,,,
0,99999,zip,98847,WA,,Chelan County,Peshastin
1,99998,zip,98846,WA,,Okanogan County,None
2,99997,zip,98845,WA,,Douglas County,Palisades
3,99996,zip,98844,WA,,Okanogan County,None
4,99995,zip,98843,WA,,Douglas County,Orondo
...,...,...,...,...,...,...,...
79240,100004,zip,98852,WA,,Okanogan County,None
79241,100003,zip,98851,WA,,Grant County,Soap Lake
79242,100002,zip,98850,WA,,Douglas County,Rock Island


In [55]:
zillow_reg[zillow_reg['region_type'] == 'neigh']

,region_id,region_type,zip,state,neigh,county,city
None,,,,,,,
625,9926,neigh,,GA,,Chatham County,Savannah
2308,9682,neigh,,LA,,,Ouachita Parish
2545,9646,neigh,,CT,,Windham County,Worcester
2618,9632,neigh,,PA,,Allegheny County,Pittsburgh
3119,9545,neigh,,NY,Albany-Schenectady-Troy,Albany County,Westmere
...,...,...,...,...,...,...,...
78862,102086,neigh,,CA,San Francisco-Oakland-Hayward,Alameda County,Central
78864,102066,neigh,,AK,,Fairbanks North Star Borough,Fairbanks
78865,102065,neigh,,AK,,Anchorage Borough,Anchorage


## Step 2: Transformation

### Creating database in PostgresQL using psycopg2
Using the DDL to create tables (talk about staging, goal)

First, we need to create staging tables as a buffer in the database to hold newly extracted data. 
I create three staging tables (data, indicator, region). The creation of the extra columns in the region table requires the creation of additional database tables for querying. Tables are created using Data Description Language (DDL). Tables are related by the primary or foreign keys. The embedded LucidChart figure illustrates the relationships among tables in this project. 

Also, we need to connect to PostgreSL using psycopg2. 

In [ ]:
##### DDL for table creation ####

def create_tables():
    
    '''create tables in postgreSQL'''
    
    commands = (
    '''
    CREATE TABLE stage_zillow_indicator(
    stage_indicator_id CHAR(5),
    stage_indicator VARCHAR(255),
    stage_category VARCHAR(255)
                                        )
    ''',
    '''
    CREATE TABLE state_zillow_region(
    stage_region_id INTEGER,
    stage_region_type VARCHAR(255),
    stage_region VARCHAR(255),
    stage_zipcode CHAR(10),
    stage_county VARCHAR (255),
    stage_city VARCHAR(255),
    stage_neigh VARCHAR(255)
                                     )
    ''',
    '''
    CREATE TABLE stage_zillow_data(
    stage_indicator_id CHAR(5),
    stage_region_id INTEGER,
    stage_date DATE,
    stage value MONEY
                                   )
    ''',
    '''
    CREATE TABLE z_indicator(
    z_ind_id SERIAL PRIMARY KEY,
    indicator_name CHAR(5) NOT NULL,
    indicator VARCHAR(255) NOT NULL,
    category_id SERIAL REFERENCES category(category_id)
                             )
    ''',
    '''
    CREATE TABLE region_type(
    region_type_id SERIAL PRIMARY KEY,
    region_type_name VARCHAR(255) NOT NULL
                             )
    ''',
    '''
    CREATE TABLE region(
    region_id SERIAL PRIMARY KEY,
    region_num INTEGER, 
    region_type_id SERIAL REFERENCES region_type(region_type_id),
    neigh VARCHAR(255),
    city VARCHAR(255),
    county VARCHAR(255),
    state CHAR(5),
    zip CHAR(10)
                       )
    ''',
    '''
    CREATE TABLE zillow_data(
    z_data SERIAL PRIMARY KEY,
    region_id SERIAL REFERENCES region(region_id),
    z_ind_id SERIAL REFFERENCES z_indicator(z_ind_id),
    z_date DATE,
    value MONEY
                             )
    ''')
    
    conn = None
    
    
    try:
        conn = psycopyg2.connect(database='zillow', user = 'postgres', password = 'postgresql', host = '127.0.0.1', port = '5432')
        print('Database connected')
        
        cur = conn.cursor()
        print('cursor')
        
        for command in commands: 
            cur.execute(command)
            
        print('Tables created')
        
        cur.close()
        print('Connection closed')
        
        conn.commit()
        print('Tables committed')
        
    except:
        print('Error loading')
        
        
#if __name__ == '__main__':
#    create tables()


## Step 3: Loading the data

### Connecting to postgresQL and loading data

## Step 4: Illustration of data model (perhaps this goes before transformation)

## Step 5: Query the database